In [9]:

from models.WAVENET.wavenet_v1.wavenet_training import generate_audio, WavenetTrainer
from safetensors import torch
from models.WAVENET.wavenet_v1.audio_data import WavenetDataset
from models.WAVENET.wavenet_v1.wavenet_model import WaveNetModel, load_latest_model_from
import tensorflow as tf

# modified WAVENET file implementation from: https://github.com/Vichoko/pytorch-wavenet/tree/master

dtype = torch.FloatTensor  # data type
ltype = torch.LongTensor  # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512,
                     output_length=16,
                     dtype=dtype,
                     bias=True)
model.cuda()
print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

data = WavenetDataset(dataset_file='./example.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='../unpacked_data',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')


def generate_and_log_samples(step):
    sample_length = 32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    # logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    # logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='birdset_model',
                         snapshot_interval=1000,
                         dtype=dtype,
                         ltype=ltype)

start_data = data[250000][0]
display(data)
trainer.train(batch_size=16,
              epochs=12)

use gpu
model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0-29): 30 x Conv1d(32, 32, kernel_size=(2,), stride=(1,))
  )
  (gate_convs): ModuleList(
    (0-29): 30 x Conv1d(32, 32, kernel_size=(2,), stride=(1,))
  )
  (residual_convs): ModuleList(
    (0-29): 30 x Conv1d(32, 32, kernel_size=(1,), stride=(1,))
  )
  (skip_convs): ModuleList(
    (0-29): 30 x Conv1d(32, 1024, kernel_size=(1,), stride=(1,))
  )
  (start_conv): Conv1d(256, 32, kernel_size=(1,), stride=(1,))
  (end_conv_1): Conv1d(1024, 512, kernel_size=(1,), stride=(1,))
  (end_conv_2): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
)
receptive field:  3070
parameter count:  1834592
one hot input
the dataset has 255490 items
start training...


epoch 0
loss at step 50: 4.003303761482239
one training step does take approximately 0.2564183950424194 seconds)
loss at step 100: 3.17235077381134
loss at step 150: 3.122062883377075
loss at step 200: 3.1990980386734007


100%|██████████| 32/32 [00:02<00:00, 10.73it/s]


validation loss: 2.68060103058815
validation accuracy: 20.2392578125%
loss at step 250: 3.056705141067505
loss at step 300: 2.9848251390457152
loss at step 350: 2.883783016204834
loss at step 400: 2.9968074655532835


100%|██████████| 32/32 [00:02<00:00, 10.94it/s]


...

100%|██████████| 32/32 [00:02<00:00, 10.89it/s]


validation loss: 1.77762046828866
validation accuracy: 36.4990234375%
loss at step 9450: 2.1047444939613342
loss at step 9500: 2.1071203541755676
loss at step 9550: 2.186663565635681
loss at step 9600: 2.1067179012298585


100%|██████████| 32/32 [00:02<00:00, 10.85it/s]


validation loss: 1.7851886972784996
validation accuracy: 36.3037109375%
loss at step 9650: 2.1217751789093016
loss at step 9700: 2.108080413341522


KeyboardInterrupt: 

In [12]:

from models.WAVENET.wavenet_v1.audio_data import WavenetDataset

dtype = torch.FloatTensor  # data type
ltype = torch.LongTensor  # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor
device = torch.device("cuda")

model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512,
                     output_length=16,
                     dtype=dtype,
                     bias=True)
model.load_state_dict(torch.load('birdset_modelwavenet_model.pth', weights_only=False))
model.eval()
model.cuda()

data = WavenetDataset(dataset_file='./example.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='./unpacked_data',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')



start_data = data[250000][0]
start_data = torch.max(start_data, 0)[1].to('cuda')
def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

generated = model.generate_fast(num_samples=64000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

use gpu
one hot input
the dataset has 255490 items
torch.Size([1, 256, 1])
0% generated
1% generated
2% generated
4% generated
one generating step does take approximately 0.011269981861114503 seconds)
5% generated
7% generated
8% generated
10% generated
11% generated
13% generated
14% generated
16% generated
17% generated
19% generated
20% generated
22% generated
23% generated
25% generated
26% generated
28% generated
29% generated
31% generated
32% generated
34% generated
35% generated
37% generated
38% generated
40% generated
41% generated
43% generated
44% generated
46% generated
47% generated
49% generated
50% generated
52% generated
53% generated
55% generated
56% generated
58% generated
59% generated
61% generated
62% generated
64% generated
65% generated
67% generated
68% generated
70% generated
71% generated
73% generated
74% generated
76% generated
77% generated
79% generated
80% generated
81% generated
83% generated
84% generated
86% generated
87% generated
89% generated
90% 

In [13]:
import IPython.display as ipd
import soundfile as sf

print(generated.shape)
ipd.Audio(generated, rate=16000)

sf.write('output_file.wav', generated, 16000)

(64000,)
